In [1]:
import pandas as pd
import numpy as np
import logging
from typing import List, Dict, Tuple

class DataBiasDetection:
    def __init__(self, data: pd.DataFrame):
        """Initialize with dataset."""
        self.data = data
        self.bias_report = {}

    def data_slicing(self, slice_cols: List[str]) -> Dict[str, pd.DataFrame]:
        """Slice data based on unique values in specified columns."""
        sliced_data = {}
        for col in slice_cols:
            unique_vals = self.data[col].unique()
            for val in unique_vals:
                slice_name = f"{col}_{val}"
                sliced_data[slice_name] = self.data[self.data[col] == val]
                logging.info(f"Data slice created: {slice_name} with {len(sliced_data[slice_name])} rows.")
        return sliced_data

    def calculate_statistics(self, sliced_data: Dict[str, pd.DataFrame], metric_col: str) -> Dict[str, float]:
        """Calculate mean statistics for each data slice."""
        slice_statistics = {}
        for slice_name, df_slice in sliced_data.items():
            mean_value = df_slice[metric_col].mean()
            slice_statistics[slice_name] = mean_value
            logging.info(f"Mean {metric_col} for slice {slice_name}: {mean_value:.2f}")
        return slice_statistics

    def detect_bias(self, slice_statistics: Dict[str, float], threshold_ratio: float = 0.2) -> List[Tuple[str, float]]:
        """Detect bias by identifying slices with significant mean deviation.
        Returns the biased slices and their mean values if bias is detected.
        """
        overall_mean = np.mean(list(slice_statistics.values()))
        biased_slices = [
            (slice_name, mean_value) for slice_name, mean_value in slice_statistics.items()
            if abs(mean_value - overall_mean) > threshold_ratio * overall_mean
        ]
        
        # Log bias detection
        if biased_slices:
            logging.warning(f"Bias detected in slices: {[slice[0] for slice in biased_slices]}")
            self.bias_report['biased_slices'] = biased_slices
        else:
            logging.info("No significant bias detected.")
            self.bias_report['biased_slices'] = []

        return biased_slices

    def document_bias_report(self) -> None:
        """Log and document bias detection results."""
        logging.info("Bias Report:")
        for key, value in self.bias_report.items():
            logging.info(f"{key}: {value}")


In [7]:
import pandas as pd
import logging

# Set up logging to see warnings and info
logging.basicConfig(level=logging.INFO)

# Load the dataset from a local CSV file
data = pd.read_csv('data_preprocess.csv')

# Instantiate the detector with the loaded data
detector = DataBiasDetection(data)

# Define columns to slice the data and the metric columns for bias detection
slice_columns = ['zone']
exclude_columns = ['datetime'] + slice_columns 
metric_columns = data.columns.difference(exclude_columns)

# Perform bias detection for each metric column
for metric_column in metric_columns:
    print(f"\nDetecting bias for metric column: {metric_column}")
    
    # Slice the data based on the slice columns
    sliced_data = detector.data_slicing(slice_cols=slice_columns)
    
    # Calculate statistics (mean) for each slice based on the metric column
    slice_statistics = detector.calculate_statistics(sliced_data, metric_col=metric_column)
    
    # Detect bias in the slices
    biased_slices = detector.detect_bias(slice_statistics)
    
    # Check and print results
    if biased_slices:
        print("Bias detected in slices:", biased_slices)
    else:
        print("No significant bias detected.")
    
    # Document the bias report for this metric column
    detector.document_bias_report()



Detecting bias for metric column: FeelsLikeF


Bias detected in slices: [('zone_8', 0.7207796383706702), ('zone_14', 0.7534238343198222)]

Detecting bias for metric column: HeatIndexF


Bias detected in slices: [('zone_8', 0.6834848596720436), ('zone_14', 0.7193004867831224)]

Detecting bias for metric column: WindChillF


Bias detected in slices: [('zone_14', 0.7648856001061237)]

Detecting bias for metric column: cloudcover


Bias detected in slices: [('zone_2', 0.5763552304018698), ('zone_16', 0.539478837794555), ('zone_19', 0.5573463738274994), ('zone_20', 0.5994930221917182), ('zone_17', 0.5557666437886069), ('zone_18', 0.5725323724548158), ('zone_10', 0.21583426987337978), ('zone_11', 0.3405769086686146), ('zone_12', 0.2923428485802053), ('zone_15', 0.26036512574611026)]

Detecting bias for metric column: humidity_lag_2
No significant bias detected.

Detecting bias for metric column: humidity_lag_4
No significant bias detected.

Detecting bias for metric column: humidity_lag_6
No significant bias detected.

Detecting bias for metric column: humidity_rolling_mean
No significant bias detected.

Detecting bias for metric column: humidity_rolling_std


...

Bias detected in slices: [('zone_8', 0.2562523413501161), ('zone_11', 0.25203166754077067), ('zone_13', 0.2544729152618566), ('zone_9', 0.25389725531330387)]

Detecting bias for metric column: windspeedMiles_lag_4


Bias detected in slices: [('zone_8', 0.25353949701555906), ('zone_11', 0.2519348917359706), ('zone_12', 0.2486882195749357), ('zone_13', 0.2525068055243376), ('zone_9', 0.25244811568142655)]

Detecting bias for metric column: windspeedMiles_lag_6


Bias detected in slices: [('zone_8', 0.2513417497065508), ('zone_10', 0.2495117504557828), ('zone_11', 0.25037274293848805), ('zone_12', 0.24983142279163856), ('zone_13', 0.25075110511725485), ('zone_14', 0.2492407782023426), ('zone_15', 0.2495629479783222), ('zone_9', 0.25045203666242105)]

Detecting bias for metric column: windspeedMiles_rolling_mean


Bias detected in slices: [('zone_8', 0.34501035809351915), ('zone_10', 0.35994599246438147), ('zone_11', 0.3495924538981095), ('zone_12', 0.3552105946748957), ('zone_13', 0.347900477481996), ('zone_14', 0.35687838204794337), ('zone_15', 0.3578241107893035), ('zone_9', 0.3468740138836967)]

Detecting bias for metric column: windspeedMiles_rolling_std


Bias detected in slices: [('zone_7', 0.15008775933597687), ('zone_3', 0.15169359822082254), ('zone_4', 0.1498716755370345), ('zone_8', 0.22909281752363828), ('zone_10', 0.23544840838450834), ('zone_14', 0.23334525237208112), ('zone_15', 0.22863126867647662)]
